In [1]:
import litebird_sim as lbs
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import warnings

%matplotlib inline
plt.rcParams.update({"font.size": 16})
warnings.filterwarnings("ignore")

In [2]:
# general
nside = 64
n_months = 3
sim_days = 30 * n_months
channel = "M1-100"
instrument = channel[0] + "FT"
sampling = 1  # this replaces channelinfo.sampling_rate_hz
imo_version = "vPTEP"  # "v1.3"

# simulation parameters
base_path = "./HWP_tutorial"
start_time = "2030-04-01T00:00:00"

# hwp_sys parameters
integrate_in_band = "true"
integrate_in_band_solver = "true"
built_map_on_the_fly = "true"
correct_in_solver = "true"
# del hwp_sys


# for band-integration, HWP systematics are passed through txt files
# order of the fields: frequency[GHz] h1 h2 beta[deg] z1 z2
# remember to set integrate_in_band and integrate_in_band_solver to 'true'
band_filename = "./HWP_systematics.txt"  # tod filling
band_filename_solver = "./HWP_systematics.txt"  # map-making

In [3]:
# creation of a parameter (.toml) file
toml_filename = "./HWP_tutorial.toml"
with open(toml_filename, "w") as f:
    f.write("[general]\n")  # --------------- GENERAL ---------------
    f.write("imo_version       = '" + imo_version + "'\n")
    f.write("instrument        = '" + instrument + "'\n")
    f.write("channel           = '" + channel + "'\n")
    f.write("nside             = " + str(nside) + "\n")
    f.write("mission_time_days = '" + str(sim_days) + "'\n\n")

    f.write("[simulation]\n")  # --------------- SIMULATION ---------------
    f.write("base_path  = '" + base_path + "'\n")
    f.write("start_time = '" + start_time + "'\n")
    f.write("duration_s = '" + str(sim_days) + " days'\n")
    f.write("name       = '" + "HWP_tutorial'\n\n")

    f.write("[hwp_sys]\n")  # --------------- HWP_SYS ---------------
    f.write(f"integrate_in_band        = {integrate_in_band}\n")
    f.write(f"integrate_in_band_solver = {integrate_in_band_solver}\n")
    f.write(f"built_map_on_the_fly     = {built_map_on_the_fly}\n")
    f.write(f"correct_in_solver        = {correct_in_solver}\n")
    f.write(
        "band_filename        = '" + band_filename + "'\n"
    )  # FUNDAMENTAL FOR READING HWP SYSTEMATICS PARAMETERS
    f.write(
        "band_filename_solver = '" + band_filename_solver + "'\n"
    )  # FUNDAMENTAL FOR READING HWP SYSTEMATICS PARAMETERS
    f.close()

In [4]:
imo = lbs.Imo(flatfile_location=lbs.PTEP_IMO_LOCATION)
sim_band_int = lbs.Simulation(
    parameter_file=toml_filename,
    random_seed=0,
)

In [5]:
# emptying the tod
# obs.tod = np.zeros_like(obs.tod)
instrumentinfo = lbs.InstrumentInfo.from_imo(
    imo, f"/releases/{imo_version}/satellite/{instrument}/instrument_info"
)

sim_band_int.set_instrument(instrumentinfo)

sim_band_int.set_scanning_strategy(
    imo_url=f"/releases/{imo_version}/satellite/scanning_parameters/"
)

channelinfo = lbs.FreqChannelInfo.from_imo(
    url=f"/releases/{imo_version}/satellite/{instrument}/{channel}/channel_info",
    imo=imo,
)

hwp_radpsec = lbs.IdealHWP(
    instrumentinfo.hwp_rpm * 2 * np.pi / 60,
).ang_speed_radpsec

sim_band_int.set_hwp(lbs.IdealHWP(hwp_radpsec))
dets = []

for detname in (
    "001_003_030_00A_100_T",
    "001_003_030_00A_100_B",
):  # we choose the couple of detector at the MFT boresight
    det = lbs.DetectorInfo.from_imo(
        url=f"/releases/{imo_version}/satellite/{instrument}/{channel}/{detname}/detector_info",
        imo=imo,
    )
    det.sampling_rate_hz = sampling
    dets.append(det)

(obs,) = sim_band_int.create_observations(dets)
sim_band_int.prepare_pointings()

In [ ]:
hwp_sys = lbs.OldHwpSys(sim_band_int)

Mbsparams = lbs.MbsParameters(
    make_cmb=True,
    seed_cmb=1234,
    cmb_r=0.0,
    make_fg=False,
    gaussian_smooth=True,
    bandpass_int=False,
    maps_in_ecliptic=False,
    nside=nside,
    units="K_CMB",
)

mbs = lbs.Mbs(
    simulation=sim_band_int,
    parameters=Mbsparams,
    channel_list=channelinfo,
)
input_map = mbs.run_all()[0][channel]
# in principle one should produce as many input maps as the number of frequencies of the band_width
# for simplicity and to save memory we scan only CMB, so we can re-use the same CMB map for the band_width
hwp_sys.set_parameters(
    nside=nside,
    mueller_or_jones="jones",
    maps=[input_map, input_map, input_map],
    Channel=channelinfo,
    #     integrate_in_band=integrate_in_band,
    #     integrate_in_band_solver=integrate_in_band_solver,
    #     correct_in_solver=correct_in_solver,
    #     built_map_on_the_fly=built_map_on_the_fly,
)
# same as before for the previous 4 lines
print(
    hwp_sys.integrate_in_band,
    hwp_sys.integrate_in_band_solver,
    hwp_sys.correct_in_solver,
    hwp_sys.built_map_on_the_fly,
)
print(hwp_sys.mII)

[2025-07-09 10:02:38,940 INFO MPI#0000] generating and saving cmb simulations
[2025-07-09 10:02:38,952 INFO MPI#0000] Sigma is 0.000000 arcmin (0.000000 rad) 
[2025-07-09 10:02:38,953 INFO MPI#0000] -> fwhm is 0.000000 arcmin
[2025-07-09 10:02:39,577 INFO MPI#0000] Sigma is 16.052182 arcmin (0.004669 rad) 
[2025-07-09 10:02:39,578 INFO MPI#0000] -> fwhm is 37.800000 arcmin
[2025-07-09 10:02:39,579 INFO MPI#0000] Sigma is 0.000000 arcmin (0.000000 rad) 
[2025-07-09 10:02:39,579 INFO MPI#0000] -> fwhm is 0.000000 arcmin


True True True True
[1.1887 1.1887 1.1887]


In [ ]:
# we do not pass pointings, they are computed on the fly
# (i.e. for all time-samples but for 1 detector at a time)
# this reduces memory allocation without really slowing down the code
print(hwp_sys.maps.shape)

hwp_sys.fill_tod(
    observations=obs,
    input_map_in_galactic=True,
    # pointings=pointings,
    # hwp_angle=hwp_angle,
)

(3, 49152)


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
# again, we do not pass pointings, they are computed on the fly
m_lbs = lbs.make_binned_map(
    nside,
    [obs],
)

In [ ]:
plt.figure(figsize=(16, 10))
ranges = 30
hp.mollview(
    (m_lbs.binned_map[0] - input_map[0]) * 1e6,
    unit="$\mu$K",
    sub=(1, 3, 1),
    title="residual T",
    coord=["G"],
    min=-ranges,
    max=ranges,
)
hp.mollview(
    (m_lbs.binned_map[1] - input_map[1]) * 1e6,
    unit="$\mu$K",
    sub=(1, 3, 2),
    title="residual Q",
    coord=["G"],
    min=-ranges * 1e-1,
    max=ranges * 1e-1,
)
hp.mollview(
    (m_lbs.binned_map[2] - input_map[2]) * 1e6,
    unit="$\mu$K",
    sub=(1, 3, 3),
    title="residual U",
    coord=["G"],
    min=-ranges * 1e-1,
    max=ranges * 1e-1,
)
plt.tight_layout()